In [1]:
from calibration_kabsch import cal_transformation_kabsch, Transformation
import numpy as np
import pyrealsense2 as rs
import dt_apriltags
import cv2
import os

In [3]:
class RSCalibration:
    
    def __init__(self, intrinsic=None, tag_family='tagStandard41h12', tag_size=None):

        self.at_detector = dt_apriltags.Detector(searchpath=['apriltags'],
                               families=tag_family,
                               nthreads=1,
                               quad_decimate=1.0,
                               quad_sigma=0.0,
                               refine_edges=1,
                               decode_sharpening=0.25,
                               debug=0)
        self.intrinsic = intrinsic
        self.tag_size = tag_size
        
    def convert_depth_pixel_to_metric_coordinate(self, depth, pixel_x, pixel_y):
        X = (pixel_x - self.intrinsic.ppx) / self.intrinsic.fx * depth
        Y = (pixel_y - self.intrinsic.ppy) / self.intrinsic.fy * depth
        return X, Y, depth
        
    def get_3d_corners(self, img, depth):
        '''
        Args:
        ------
        img: (H, W, C) ndarray, 1 for gray or 3 for color.
        depth: (H, W) ndarray.
        '''
        assert img.shape[:2] == depth.shape[:2]
        if img.shape[-1] == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        elif img.shape[-1] == 1:
            gray = img
        else:
            raise Exception('Only support gray or color.')
        detections = self.at_detector.detect(gray)
        if detections:
            corners2D = {}
            for each in detections:
                corners, center = each.corners.tolist(), each.center.tolist()
                #corners.append(center)
                corners2D[each.tag_id] = corners
                
            corners3D = {}
            for k, v in corners2D.items():
                corners3D[k] = []
                for point in v:
                    X = point[0]
                    Y = point[1]
                    Z = depth[int(round(Y)), int(round(X))]
                    if Z == 0 or Z is None:
                        Z = None
                    # Convert
                    else:
                        X, Y, Z = self.convert_depth_pixel_to_metric_coordinate(Z, X, Y)
                    corners3D[k].append([X, Y, Z])
            return corners3D
        else:
            return None



def perform_RT(src3D, dst3D):
    
    srcPoints = []
    dstPoints = []
    
    for k in src3D.keys():
        if not k in dst3D.keys():
            continue
        for i in range(len(src3D[k])):
            src_point = src3D[k][i]
            dst_point = dst3D[k][i]
            # Check Z value is ok.
            
            if src_point[2] and dst_point[2]:
                srcPoints.append(src_point)
                dstPoints.append(dst_point)
    if len(srcPoints) < 5:
        return None, None

    srcPoints = np.asarray(srcPoints)
    dstPoints = np.asarray(dstPoints)
    R, T, rmsd_value = cal_transformation_kabsch(srcPoints, dstPoints)
    print(rmsd_value)
    return R, T

In [4]:
def handle_baodi(srcCam, dstCam, imgs_dir, srcIntrinsic, dstIntrinsic):
    '''
    Args:
      srcCam: str, camera's name, eg: rs15 or rs16
      dstCam: str.
      imgs_dir: str, the path of images, filename is [camName]_[seq].jpg or png
                eg: rs15_0.jpg, rs16_1.jpg...
    '''
    
    allfiles = os.listdir(imgs_dir)
    allfiles.sort()
    
    absPath = lambda file: os.path.join(imgs_dir, file) 
    srcColorFiles = [absPath(file) for file in allfiles if ('.jpg' in file and srcCam in file)]
    srcDepthFiles = [absPath(file) for file in allfiles if ('.png' in file and srcCam in file)]
    dstColorFiles = [absPath(file) for file in allfiles if ('.jpg' in file and dstCam in file)]
    dstDepthFiles = [absPath(file) for file in allfiles if ('.png' in file and dstCam in file)]
    assert len(srcColorFiles) == len(srcDepthFiles) == len(dstColorFiles) == len(dstDepthFiles)
    # get sequence length.
    loop = len(srcColorFiles)
    
    srcRS = RSCalibration(intrinsic=srcIntrinsic)
    dstRS = RSCalibration(intrinsic=dstIntrinsic)
    src3D_valid = []
    dst3D_valid = []
    
    for i in range(loop):
        srcColor, srcDepth = cv2.imread(srcColorFiles[i]), cv2.imread(srcDepthFiles[i], cv2.IMREAD_ANYDEPTH) 
        dstColor, dstDepth = cv2.imread(dstColorFiles[i]), cv2.imread(dstDepthFiles[i], cv2.IMREAD_ANYDEPTH)
        
        src3D = srcRS.get_3d_corners(srcColor, srcDepth)
        dst3D = dstRS.get_3d_corners(dstColor, dstDepth)
        
        if src3D and dst3D:
            for k in src3D.keys():
                if not k in dst3D.keys():
                    continue
                for i in range(len(src3D[k])):
                    src_point = src3D[k][i]
                    dst_point = dst3D[k][i]
                    
                    if src_point[2] and dst_point[2]:
                        src3D_valid.append(src_point)
                        dst3D_valid.append(dst_point)
    assert len(src3D_valid) > 5                    
    srcPoints = np.asarray(src3D_valid)
    dstPoints = np.asarray(dst3D_valid)
    R, T, rmsd_value = cal_transformation_kabsch(srcPoints, dstPoints)
    print(rmsd_value)
    return R, T

In [5]:
import json
from easydict import EasyDict
intrinsics_file = '/home/commaai-03/Data/color/apriltags/intrinsics.json'
with open(intrinsics_file, 'r') as f:
    intrinsics = json.load(f)
    intrinsics = EasyDict(intrinsics)
    
imgs_dir = '/home/commaai-03/Data/color/apriltags'

In [7]:
R, T = handle_baodi(srcCam='rs15',
             dstCam='rs16',
             imgs_dir=imgs_dir,
             srcIntrinsic=intrinsics.rs15,
             dstIntrinsic=intrinsics.rs16)

R, T

80.72483309446866


(array([[ 9.89204577e-01,  5.11590156e-02,  1.37321014e-01],
        [-5.07422327e-02,  9.98690394e-01, -6.53628019e-03],
        [-1.37475567e-01, -5.02256558e-04,  9.90505031e-01]]),
 array([-626.0794935 ,  884.22278229,  177.23313627]))

In [ ]:
80.74263480130824
(array([[ 9.89177226e-01,  5.16229793e-02,  1.37344398e-01],
        [-5.12005139e-02,  9.98666523e-01, -6.60936323e-03],
        [-1.37502448e-01, -4.94272171e-04,  9.90501304e-01]]),
 array([-625.7165647 ,  885.27609408,  177.27231593]))

In [145]:
i15, d15 = imgs_dir + '/rs15_0.jpg', imgs_dir + '/rs15_0.png'
i15, d15 = cv2.imread(i15), cv2.imread(d15, cv2.IMREAD_ANYDEPTH)
i16, d16 = imgs_dir + '/rs16_0.jpg', imgs_dir + '/rs16_0.png'
i16, d16 = cv2.imread(i16), cv2.imread(d16, cv2.IMREAD_ANYDEPTH)

RS15, RS16 = RSCalibration(intrinsic=intrinsics.rs15), RSCalibration(intrinsic=intrinsics.rs16)

p15 = RS15.get_3d_corners(i15, d15)
p16 = RS16.get_3d_corners(i16, d16)

In [13]:
p16

{9: [[1029.4184478426162, 451.72074449690683, 2975],
  [943.9195528149374, 450.77555067148035, 2962],
  [940.4149670483158, 524.7050845479765, 2936],
  [991.1168810534929, 511.1152791459044, 2861]],
 11: [[778.647337333774, 464.1278097941673, 3001],
  [686.8233330419037, 454.81062953063196, 2949],
  [700.0256470283048, 542.9431179944609, 3001],
  [769.7090710938467, 535.7089483452044, 2962]],
 21: [[1004.57047196792, 171.8188122410669, 2910],
  [939.5566830605551, 174.0239472472308, 2949],
  [927.7997489858564, 249.30320518144555, 2910],
  [988.9207469531513, 245.7303121530997, 2861]],
 99: [[770.3026122678958, 168.18852553040387, 2975],
  [694.0718493502296, 168.74942285657613, 2988],
  [703.4889132111705, 250.49079552367905, 3028],
  [786.4584645226429, 251.89752165460013, 3042]]}

In [12]:
cv2.imshow('rs15', i15)
cv2.imshow('rs16', i16)
k = cv2.waitKey()
if k:
    cv2.destroyAllWindows()

In [14]:
pose_file = '/home/commaai-03/Public/ye/camera_poses.yaml'
import yaml
with open(pose_file, 'r') as f:
    yaml_pose = yaml.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [218]:
poses = EasyDict(yaml_pose['poses'])

In [207]:
def q2r(x, y, z, w):
    x2, y2, z2, w2 = np.square(x), np.square(y), np.square(z), np.square(w)
    xy = x * y
    zw = z * w
    xz = x * z
    yw = y * w
    yz = y * z
    xw = x * w
    #assert np.abs(x2 + y2 + z2 + w2 == 1
    R = [[x2 - y2 - z2 + w2, 2 * (xy + zw), 2 * (xz - yw)],
         [2 * (xy - zw), -x2 + y2 - z2 + w2, 2 * (yz + xw)],
         [2 * (xz + yw), 2 * (yz - xw), -x2 - y2 + z2 + w2]
        ]
    R = np.asarray(R)
    return R

In [219]:
x, y, z, w = [poses.rs15.rotation.x,
              poses.rs15.rotation.y,
              poses.rs15.rotation.z,
              poses.rs15.rotation.w]
r15 = q2r(x, y, z, w)
t15 = np.asarray([poses.rs15.translation.x, 
                  poses.rs15.translation.y,
                  poses.rs15.translation.z])

x, y, z, w = [poses.rs16.rotation.x,
              poses.rs16.rotation.y,
              poses.rs16.rotation.z,
              poses.rs16.rotation.w]
r16 = q2r(x, y, z, w)
t16 = np.asarray([poses.rs16.translation.x, 
                  poses.rs16.translation.y,
                  poses.rs16.translation.z])

In [220]:
mat15 = np.zeros((4,4))
mat15[:3,:3] = r15
mat15[:3,3] = t15
mat15[3,3] = 1.0

mat16 = np.zeros((4,4))
mat16[:3,:3] = r16
mat16[:3,3] = t16
mat16[3,3] = 1.0

In [221]:
mat15

array([[ 0.02232187, -0.99886687, -0.04203228, -0.31349071],
       [-0.99899794, -0.02065399, -0.03970555,  1.05958542],
       [ 0.03879242,  0.04287646, -0.99832698,  3.08604742],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [237]:
mat16

array([[-0.02412933, -0.99969946,  0.00433129,  0.58678025],
       [-0.99912732,  0.02396731, -0.0342077 ,  0.9619321 ],
       [ 0.03409361, -0.00515291, -0.99940536,  3.08382652],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [223]:
p1 = np.asarray(p15[9]) / 1000.
p2 = np.asarray(p16[9]) / 1000.

In [234]:
np.dot(mat16, [0,0,0,1])

array([0.87011248, 0.57943966, 3.11235672, 1.        ])

In [224]:
p1

array([[ 1.25800051, -0.3594974 ,  2.917     ],
       [ 1.1762206 , -0.35972738,  2.917     ],
       [ 1.16853385, -0.27896083,  2.904     ],
       [ 1.25023226, -0.27820992,  2.904     ]])

In [225]:
p2

array([[1.02941845, 0.45172074, 2.975     ],
       [0.94391955, 0.45077555, 2.962     ],
       [0.94041497, 0.52470508, 2.936     ],
       [0.99111688, 0.51111528, 2.861     ]])

In [226]:
def point2Homo(points):
    N = len(points)
    tmp = np.empty((N,4))
    tmp[:N, :3] = points
    tmp[:, 3] = 1.
    return tmp

In [227]:
#np.dot(mat15, point2Homo(p1))

p1_w = np.asarray([np.dot(mat15, point) for point in point2Homo(p1)])
p1_w

array([[-0.04892789, -0.30555053,  0.20731454,  1.        ],
       [-0.05052365, -0.22384781,  0.20413223,  1.        ],
       [-0.13082385, -0.21732075,  0.22027528,  1.        ],
       [-0.12975025, -0.2989528 ,  0.22347676,  1.        ]])

In [228]:
#np.dot(mat16, point2Homo(p2))

p2_w = np.asarray([np.dot(mat16, point) for point in point2Homo(p2)])
p2_w

array([[ 0.12324166, -0.15752937,  0.14336449,  1.        ],
       [ 0.12619329, -0.07168304,  0.15344666,  1.        ],
       [ 0.05225793, -0.06552022,  0.17893076,  1.        ],
       [ 0.0642954 , -0.11393802,  0.2556848 ,  1.        ]])

In [160]:
from scipy.spatial.transform import Rotation as R

r = R.from_quat([x, y, z, w])

In [161]:
r16 = r.as_dcm()

In [98]:
r16

array([[-0.02412933, -0.99912732,  0.03409361],
       [-0.99969946,  0.02396731, -0.00515291],
       [ 0.00433129, -0.0342077 , -0.99940536]])

In [186]:
diff = p1_w[:,:3]/1000. - p2_w[:,:3]/1000.
N = len(p1)
np.sqrt((diff * diff).sum() / N)

0.37280028049995634

In [189]:
p1_w[:,:3]/1000.

array([[-0.04892789, -0.30555053,  0.20731454],
       [-0.05052365, -0.22384781,  0.20413223],
       [-0.13082385, -0.21732075,  0.22027528],
       [-0.12975025, -0.2989528 ,  0.22347676]])

In [188]:
p2_w/1000.

array([[ 0.21204302, -0.07168036,  0.09960195,  0.001     ],
       [ 0.2146072 ,  0.01383717,  0.11225624,  0.001     ],
       [ 0.13994031,  0.01924657,  0.13569664,  0.001     ],
       [ 0.14973783, -0.03137934,  0.21133652,  0.001     ]])